<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Objectifs" data-toc-modified-id="Objectifs-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Objectifs</a></span></li><li><span><a href="#Dev" data-toc-modified-id="Dev-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Dev</a></span><ul class="toc-item"><li><span><a href="#Données-et-preprocessing" data-toc-modified-id="Données-et-preprocessing-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>Données et preprocessing</a></span></li><li><span><a href="#Plotly" data-toc-modified-id="Plotly-2.2"><span class="toc-item-num">2.2&nbsp;&nbsp;</span>Plotly</a></span></li><li><span><a href="#Kepler" data-toc-modified-id="Kepler-2.3"><span class="toc-item-num">2.3&nbsp;&nbsp;</span>Kepler</a></span><ul class="toc-item"><li><span><a href="#Preprocessing-data-kepler" data-toc-modified-id="Preprocessing-data-kepler-2.3.1"><span class="toc-item-num">2.3.1&nbsp;&nbsp;</span>Preprocessing data kepler</a></span></li><li><span><a href="#Graphique" data-toc-modified-id="Graphique-2.3.2"><span class="toc-item-num">2.3.2&nbsp;&nbsp;</span>Graphique</a></span></li></ul></li></ul></li><li><span><a href="#Industrialisation" data-toc-modified-id="Industrialisation-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Industrialisation</a></span><ul class="toc-item"><li><span><a href="#Simulation-de-données" data-toc-modified-id="Simulation-de-données-3.1"><span class="toc-item-num">3.1&nbsp;&nbsp;</span>Simulation de données</a></span></li><li><span><a href="#Avec-plotly" data-toc-modified-id="Avec-plotly-3.2"><span class="toc-item-num">3.2&nbsp;&nbsp;</span>Avec plotly</a></span></li><li><span><a href="#Avec-Kepler" data-toc-modified-id="Avec-Kepler-3.3"><span class="toc-item-num">3.3&nbsp;&nbsp;</span>Avec Kepler</a></span></li></ul></li></ul></div>

In [1]:
import pandas as pd
import numpy as np
import io

from vcub_keeper.reader.reader import *
from vcub_keeper.transform.features_factory import *
from vcub_keeper.visualisation import plot_station_activity
from vcub_keeper.config import MAPBOX_TOKEN, THRESHOLD_PROFILE_STATION

import plotly.express as px
import plotly.graph_objs as go
from plotly.offline import init_notebook_mode, iplot, offline
from keplergl import KeplerGl

import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style('whitegrid')
%matplotlib inline

%load_ext autoreload
%autoreload 2

# Objectifs

- Création d'une map afin de connaitre les stations actives, inactives et HS (d'après le résultat de l'algo).
- Prendre en compte la structure des données en INPUT pour que cela puisse marcher avec les données de `station_control.csv` (Projet `vcub_watcher` ).
- Issue gitub : https://github.com/armgilles/vcub_keeper/issues/49

# Dev

## Données et preprocessing

In [2]:
# Simulation de données

simulated_data="""station_id,mean_activity,is_anomaly,is_inactive,last_date_anomaly,anomaly_since
6,0.4,1,0,2021-04-07 11:10:00+02:00,2021-04-07 09:00:00+02:00
59,0.42,1,0,2021-04-07 11:10:00+02:00,2021-04-07 10:10:00+02:00
138,0.36,0,1,,
181,0.0,0,0,,
181,0.0,0,0,,
160,0.0,0,0,,
161,0.01,0,0,,
180,0.01,0,0,,
92,0.01,0,0,,
167,0.01,0,0,,
168,0.01,0,0,,
183,0.01,0,0,,
81,0.02,0,0,,
150,0.02,0,0,,
72,0.02,0,0,,
71,0.02,0,0,,
91,0.02,0,0,,
80,0.02,0,0,,
165,0.02,0,0,,
156,0.02,0,0,,
157,0.03,0,0,,
87,0.03,0,0,,
182,0.03,0,0,,
93,0.03,0,0,,
94,0.03,0,0,,
149,0.04,0,0,,
82,0.04,0,0,,
88,0.05,0,0,,
164,0.05,0,0,,
76,0.05,0,0,,
77,0.05,0,0,,
158,0.05,0,0,,
148,0.05,0,0,,
79,0.05,0,0,,
95,0.05,0,0,,
151,0.05,0,0,,
86,0.06,0,0,,
153,0.06,0,0,,
159,0.06,0,0,,
85,0.06,0,0,,
162,0.06,0,0,,
169,0.07,0,0,,
177,0.07,0,0,,
78,0.07,0,0,,
147,0.07,0,0,,
97,0.07,0,0,,
84,0.08,0,0,,
163,0.08,0,0,,
63,0.09,0,0,,
146,0.09,0,0,,
166,0.1,0,0,,
89,0.1,0,0,,
152,0.1,0,0,,
143,0.1,0,0,,
115,0.1,0,0,,
142,0.11,0,0,,
64,0.11,0,0,,
74,0.12,0,0,,
144,0.12,0,0,,
51,0.12,0,0,,
114,0.13,0,0,,
145,0.13,0,0,,
83,0.13,0,0,,
113,0.13,0,0,,
251,0.13,0,0,,
38,0.13,0,0,,
70,0.14,0,0,,
75,0.14,0,0,,
175,0.14,0,0,,
178,0.16,0,0,,
32,0.16,0,0,,
12,0.17,0,0,,
90,0.17,0,0,,
50,0.17,0,0,,
49,0.18,0,0,,
155,0.18,0,0,,
129,0.18,0,0,,
52,0.18,0,0,,
26,0.18,0,0,,
31,0.18,0,0,,
29,0.19,0,0,,
53,0.2,0,0,,
30,0.2,0,0,,
154,0.2,0,0,,
35,0.2,0,0,,
126,0.2,0,0,,
176,0.21,0,0,,
25,0.21,0,0,,
117,0.21,0,0,,
122,0.21,0,0,,
137,0.21,0,0,,
96,0.22,0,0,,
73,0.23,0,0,,
141,0.23,0,0,,
27,0.23,0,0,,
69,0.23,0,0,,
14,0.23,0,0,,
179,0.23,0,0,,
47,0.24,0,0,,
13,0.24,0,0,,
46,0.24,0,0,,
116,0.24,0,0,,
48,0.24,0,0,,
67,0.25,0,0,,
121,0.25,0,0,,
173,0.25,0,0,,
17,0.26,0,0,,
140,0.27,0,0,,
107,0.27,0,0,,
171,0.27,0,0,,
33,0.27,0,0,,
61,0.27,0,0,,
34,0.27,0,0,,
170,0.28,0,0,,
132,0.28,0,0,,
128,0.28,0,0,,
119,0.28,0,0,,
15,0.29,0,0,,
18,0.29,0,0,,
124,0.3,0,0,,
118,0.3,0,0,,
111,0.31,0,0,,
10,0.31,0,0,,
62,0.32,0,0,,
66,0.32,0,0,,
105,0.32,0,0,,
130,0.32,0,0,,
112,0.32,0,0,,
2,0.32,0,0,,
68,0.32,0,0,,
23,0.34,0,0,,
11,0.34,0,0,,
60,0.34,0,0,,
120,0.34,0,0,,
9,0.34,0,0,,
3,0.35,0,0,,
45,0.35,0,0,,
136,0.36,0,0,,
24,0.36,0,0,,
21,0.36,0,0,,
108,0.37,0,0,,
16,0.37,0,0,,
131,0.37,0,0,,
36,0.38,0,0,,
20,0.39,0,0,,
110,0.39,0,0,,
139,0.39,0,0,,
42,0.4,0,0,,
125,0.4,0,0,,
19,0.41,0,0,,
135,0.41,0,0,,
41,0.43,0,0,,
28,0.44,0,0,,
7,0.44,0,0,,
109,0.44,0,0,,
4,0.44,0,0,,
57,0.44,0,0,,
55,0.44,0,0,,
37,0.45,0,0,,
8,0.46,0,0,,
58,0.47,0,0,,
99,0.48,0,0,,
134,0.48,0,0,,
104,0.49,0,0,,
40,0.49,0,0,,
98,0.49,0,0,,
172,0.5,0,0,,
1,0.51,0,0,,
56,0.51,0,0,,
43,0.51,0,0,,
44,0.53,0,0,,
100,0.54,0,0,,
133,0.55,0,0,,
103,0.58,0,0,,
101,0.6,0,0,,
102,0.63,0,0,,
5,0.64,0,0,,
65,0.65,0,0,,
174,0.67,0,0,,
123,0.74,0,0,,
54,0.74,0,0,,
39,0.82,0,0,,
22,0.82,0,0,,
127,0.9,0,0,,
106,1.03,0,0,,
"""

station_control = pd.read_csv( io.StringIO(simulated_data)  , sep=",")

# Idem as read_station_control_from_s3 from vcub_watcher functino
station_control['last_date_anomaly'] = \
        pd.to_datetime(station_control['last_date_anomaly'])
try:
    station_control['last_date_anomaly'] = \
        station_control['last_date_anomaly'].dt.tz_localize('Europe/Paris')
except:  # try to convert TZ
    station_control['last_date_anomaly'] = \
        station_control['last_date_anomaly'].dt.tz_convert('Europe/Paris')

#station_control['anomaly_since'] = \
#    pd.to_datetime(station_control['anomaly_since'], utc=True)
station_control['anomaly_since'] = \
    pd.to_datetime(station_control['anomaly_since'])
try:
    station_control['anomaly_since'] = \
        station_control['anomaly_since'].dt.tz_localize('Europe/Paris')
except:  # try to convert TZ
    station_control['anomaly_since'] = \
        station_control['anomaly_since'].dt.tz_convert('Europe/Paris')
    
# Simulate "available_bikes" - Nombre de vélo présent à la station
station_control['available_bikes'] = np.random.randint(0, 40, len(station_control))

In [3]:
station_control

,station_id,mean_activity,is_anomaly,is_inactive,last_date_anomaly,anomaly_since,available_bikes
0,6,0.40,1,0,2021-04-07 11:10:00+02:00,2021-04-07 09:00:00+02:00,27
1,59,0.42,1,0,2021-04-07 11:10:00+02:00,2021-04-07 10:10:00+02:00,8
2,138,0.36,0,1,NaT,NaT,35
3,181,0.00,0,0,NaT,NaT,4
4,181,0.00,0,0,NaT,NaT,29
...,...,...,...,...,...,...,...
180,54,0.74,0,0,NaT,NaT,37
181,39,0.82,0,0,NaT,NaT,9
182,22,0.82,0,0,NaT,NaT,22
183,127,0.90,0,0,NaT,NaT,30


In [4]:
# Ajouter le lat/lon à station_control
# Lecture du fichier de réf des stations
stations = read_stations_attributes(path_directory=ROOT_DATA_REF)
stations = stations[['station_id', 'lat', 'lon', 'NOM']]
stations

# Merge des données
station_control = station_control.merge(stations, on='station_id', how='left')

# On ne prend que les stations avec lat / lon
station_control = station_control[~station_control.lat.isna()]


In [5]:
station_control[station_control.lat.isna()] # Ne dois pas avoir de ligne

,station_id,mean_activity,is_anomaly,is_inactive,last_date_anomaly,anomaly_since,available_bikes,lat,lon,NOM


In [6]:
station_control

,station_id,mean_activity,is_anomaly,is_inactive,last_date_anomaly,anomaly_since,available_bikes,lat,lon,NOM
0,6,0.40,1,0,2021-04-07 11:10:00+02:00,2021-04-07 09:00:00+02:00,27,44.837790,-0.581660,Square Andre Lhote
1,59,0.42,1,0,2021-04-07 11:10:00+02:00,2021-04-07 10:10:00+02:00,8,44.849780,-0.570240,CAPC
3,181,0.00,0,0,NaT,NaT,4,44.866214,-0.605290,Mairie du Bouscat
4,181,0.00,0,0,NaT,NaT,29,44.866214,-0.605290,Mairie du Bouscat
5,160,0.00,0,0,NaT,NaT,21,44.912045,-0.724596,Eglise St Aubin
...,...,...,...,...,...,...,...,...,...,...
180,54,0.74,0,0,NaT,NaT,37,44.826420,-0.557320,Rue St Vincent de Paul
181,39,0.82,0,0,NaT,NaT,9,44.844230,-0.574340,Quinconces
182,22,0.82,0,0,NaT,NaT,22,44.838260,-0.576480,Hotel de Ville
183,127,0.90,0,0,NaT,NaT,30,44.826300,-0.557070,Gare St Jean


In [7]:
# Preprocess avant graphique
station_control['etat'] = 'normal'

# Non monitoré
station_control.loc[station_control['mean_activity'] < THRESHOLD_PROFILE_STATION,
                    'etat'] = 'non surveillée'

# En anoamlie (HS prediction)
station_control.loc[station_control['is_anomaly'] == 1, 'etat'] = 'anomaly'

# Inactive
station_control.loc[station_control['is_inactive'] == 1, 'etat'] = 'inactive'

# Transform date to string
station_control['anomaly_since_str'] = station_control['anomaly_since'].dt.strftime(date_format='%Y-%m-%d %H:%M')
station_control['anomaly_since_str'] = station_control['anomaly_since_str'].fillna('-')

In [8]:
station_control

,station_id,mean_activity,is_anomaly,is_inactive,last_date_anomaly,anomaly_since,available_bikes,lat,lon,NOM,etat,anomaly_since_str
0,6,0.40,1,0,2021-04-07 11:10:00+02:00,2021-04-07 09:00:00+02:00,27,44.837790,-0.581660,Square Andre Lhote,anomaly,2021-04-07 09:00
1,59,0.42,1,0,2021-04-07 11:10:00+02:00,2021-04-07 10:10:00+02:00,8,44.849780,-0.570240,CAPC,anomaly,2021-04-07 10:10
3,181,0.00,0,0,NaT,NaT,4,44.866214,-0.605290,Mairie du Bouscat,non surveillée,-
4,181,0.00,0,0,NaT,NaT,29,44.866214,-0.605290,Mairie du Bouscat,non surveillée,-
5,160,0.00,0,0,NaT,NaT,21,44.912045,-0.724596,Eglise St Aubin,non surveillée,-
...,...,...,...,...,...,...,...,...,...,...,...,...
180,54,0.74,0,0,NaT,NaT,37,44.826420,-0.557320,Rue St Vincent de Paul,normal,-
181,39,0.82,0,0,NaT,NaT,9,44.844230,-0.574340,Quinconces,normal,-
182,22,0.82,0,0,NaT,NaT,22,44.838260,-0.576480,Hotel de Ville,normal,-
183,127,0.90,0,0,NaT,NaT,30,44.826300,-0.557070,Gare St Jean,normal,-


## Plotly

In [9]:
texts = []
for idx, station in station_control.iterrows():
    text = str(station['NOM']) \
        + " <br />" + "station N° : " + str(station['station_id']) \
        + " <br />" + "Nombre de vélo dispo : " + str(station['available_bikes']) \
        + " <br />" + "Activité suspecte depuis : " + str(station['anomaly_since_str'])
    texts.append(text)

In [ ]:
# Plotly express

px.set_mapbox_access_token(MAPBOX_TOKEN)
fig = px.scatter_mapbox(station_control,
                        lat="lat",
                        lon="lon",
                        color='etat',
                        #hoverinfo=texts,
                        hover_data=['mean_activity'],
                        #labels={'station_id': 'station N°)',
                        #        'available_bikes': 'Nombre de vélo dispo',
                        #        'anomaly_since': 'Suspect depuis'},
                        zoom=13)
fig.show()

In [ ]:
# Plotly figure (sans état)
fig = go.Figure(go.Scattermapbox(lat=station_control['lat'],
                         lon=station_control['lon'],
                         mode='markers',
                         text=texts,
                         hoverinfo="text",
                         marker_size=7, 
                         marker_color='red'))
fig.update_layout(mapbox = dict(center= dict(lat=44.837794, lon=-0.581662),            
                                 accesstoken= MAPBOX_TOKEN,
                                 zoom=12,
                                 style="light"
                               ))

In [12]:
station_control.etat.unique()

array(['anomaly', 'non surveillée', 'normal'], dtype=object)

In [ ]:
color_etat = {'anomaly': '#EB4D50',
              'inactive': '#5E9BE6',
              'normal': '#6DDE75',
              'non surveillée' : '#696A6A'}

wtf_compteur = 0
for etat in station_control['etat'].unique():
#for etat in station_control[station_control['etat'] == 'anomaly']['etat'].unique():
    # Filter
    temp = station_control[station_control['etat'] == etat]
    
    # Building text
    texts = []
    for idx, station in temp.iterrows():
        text = str(station['NOM']) \
            + " <br />" + "station N° : " + str(station['station_id']) \
            + " <br />" + "Nombre de vélo dispo : " + str(station['available_bikes']) \
            + " <br />" + "Activité suspecte depuis : " + str(station['anomaly_since_str'])
        texts.append(text)
    
    if wtf_compteur == 0:
        fig = go.Figure(go.Scattermapbox(lat=temp['lat'],
                                         lon=temp['lon'],
                                         mode='markers',
                                         #text=texts,
                                         hoverinfo='text',
                                         hovertext=texts,
                                         marker_size=8, 
                                         marker_color=color_etat[etat],
                                         name=etat))
    else:
        fig.add_trace(go.Scattermapbox(lat=temp['lat'],
                                         lon=temp['lon'],
                                         mode='markers',
                                         #text=texts,
                                         hoverinfo='text',
                                         hovertext=texts,
                                         marker_size=8, 
                                         marker_color=color_etat[etat],
                                         name=etat))
        
    wtf_compteur = 1
    
fig.update_layout(mapbox=dict(center= dict(lat=44.837794, lon=-0.581662),            
                              accesstoken= MAPBOX_TOKEN,
                              zoom=15,
                              style="light"),
                  showlegend=True,
                  legend=dict(orientation="h",
                              yanchor="top",
                              xanchor="center",
                              y=1.1,
                              x=0.5
                    )
)

## Kepler

In [14]:
# Installation de Kepler
#!pip install keplergl

# To Do in terminal : https://github.com/keplergl/kepler.gl/tree/master/bindings/kepler.gl-jupyter#installation
#jupyter nbextension install --py --sys-prefix keplergl # can be skipped for notebook 5.3 and above
#jupyter nbextension enable --py --sys-prefix keplergl # can be skipped for notebook 5.3 and above

In [15]:
from keplergl import KeplerGl

### Preprocessing data kepler

In [16]:
station_control_kepler = station_control.drop(['last_date_anomaly', 'anomaly_since'], axis=1)

In [17]:
station_control_kepler

,station_id,mean_activity,is_anomaly,is_inactive,available_bikes,lat,lon,NOM,etat,anomaly_since_str
0,6,0.40,1,0,27,44.837790,-0.581660,Square Andre Lhote,anomaly,2021-04-07 09:00
1,59,0.42,1,0,8,44.849780,-0.570240,CAPC,anomaly,2021-04-07 10:10
3,181,0.00,0,0,4,44.866214,-0.605290,Mairie du Bouscat,non surveillée,-
4,181,0.00,0,0,29,44.866214,-0.605290,Mairie du Bouscat,non surveillée,-
5,160,0.00,0,0,21,44.912045,-0.724596,Eglise St Aubin,non surveillée,-
...,...,...,...,...,...,...,...,...,...,...
180,54,0.74,0,0,37,44.826420,-0.557320,Rue St Vincent de Paul,normal,-
181,39,0.82,0,0,9,44.844230,-0.574340,Quinconces,normal,-
182,22,0.82,0,0,22,44.838260,-0.576480,Hotel de Ville,normal,-
183,127,0.90,0,0,30,44.826300,-0.557070,Gare St Jean,normal,-


In [18]:
# Add fake stations to have every type of etat (anomaly / normal / inactive 
# & non surveillée) to match with fill color in plot

fake_station_every_etat = \
    [[990, 0.4, 1, 0, 35, 60.9616624, -39.1527227, 'Station fake anomaly',
            'anomaly', '2021-04-07 09:00'],
     [991, 0.42, 1, 0, 21, 60.9616624, -39.1527227, 'Station fake normal',
      'normal', '-'],
     [992, 0.36, 0, 1, 22, 60.9616624, -39.1527227, 'Station fake inactive',
        'inactive', '-'],
     [993, 0.0, 0, 0, 22, 60.9616624, -39.1527227, 'Station non surveillée',
        'non surveillée', '-']]

fake_station_every_etat_df = pd.DataFrame(fake_station_every_etat, columns=station_control_kepler.columns)
station_control_kepler = pd.concat([station_control_kepler, fake_station_every_etat_df])

In [19]:
# Sorting DataFrame to fill color in correct order
etat_id_sort = {'non surveillée': 0,
                    'normal': 1,
                    'anomaly': 2,
                    'inactive': 3}

station_control_kepler['etat_id_sort'] = station_control_kepler['etat'].map(etat_id_sort)
station_control_kepler = station_control_kepler.sort_values('etat_id_sort')

In [20]:
station_control_kepler.tail()

,station_id,mean_activity,is_anomaly,is_inactive,available_bikes,lat,lon,NOM,etat,anomaly_since_str,etat_id_sort
184,106,1.03,0,0,26,44.830640,-0.573210,Place de la Victoire,normal,-,1
0,6,0.40,1,0,27,44.837790,-0.581660,Square Andre Lhote,anomaly,2021-04-07 09:00,2
1,59,0.42,1,0,8,44.849780,-0.570240,CAPC,anomaly,2021-04-07 10:10,2
0,990,0.40,1,0,35,60.961662,-39.152723,Station fake anomaly,anomaly,2021-04-07 09:00,2
2,992,0.36,0,1,22,60.961662,-39.152723,Station fake inactive,inactive,-,3


In [21]:
config_global = {
  "version": "v1",
  "config": {
    "visState": {
      "filters": [],
      "layers": [
        {
          "id": "fmdzqhw",
          "type": "point",
          "config": {
            "dataId": "data_1",
            "label": "Station Vcub",
            "color": [
              18,
              147,
              154
            ],
            "columns": {
              "lat": "lat",
              "lng": "lon",
              "altitude": None
            },
            "isVisible": True,
            "visConfig": {
              "radius": 12,
              "fixedRadius": False,
              "opacity": 0.8,
              "outline": False,
              "thickness": 2,
              "strokeColor": None,
              "colorRange": {
                "name": "Custom Palette",
                "type": "custom",
                "category": "Custom",
                "colors": [
                  "#696A6A",
                  "#6DDE75",
                  "#EB4D50",
                  "#5E9BE6"
                ]
              },
              "strokeColorRange": {
                "name": "Global Warming",
                "type": "sequential",
                "category": "Uber",
                "colors": [
                  "#5A1846",
                  "#900C3F",
                  "#C70039",
                  "#E3611C",
                  "#F1920E",
                  "#FFC300"
                ]
              },
              "radiusRange": [
                0,
                50
              ],
              "filled": True
            },
            "hidden": False,
            "textLabel": [
              {
                "field": None,
                "color": [
                  255,
                  255,
                  255
                ],
                "size": 18,
                "offset": [
                  0,
                  0
                ],
                "anchor": "start",
                "alignment": "center"
              }
            ]
          },
          "visualChannels": {
            "colorField": {
              "name": "etat_id_sort",
              "type": "integer"
            },
            "colorScale": "ordinal",
            "strokeColorField": None,
            "strokeColorScale": "quantile",
            "sizeField": None,
            "sizeScale": "linear"
          }
        }
      ],
      "interactionConfig": {
        "tooltip": {
          "fieldsToShow": {
            "data_1": [
              {
                "name": "station_id",
                "format": None
              },
              {
                "name": "NOM",
                "format": None
              },
              {
                "name": "etat",
                "format": None
              },
              {
                "name": "available_bikes",
                "format": None
              },
              {
                "name": "anomaly_since_str",
                "format": None
              }
            ]
          },
          "compareMode": True,
          "compareType": "absolute",
          "enabled": True
        },
        "brush": {
          "size": 0.5,
          "enabled": False
        },
        "geocoder": {
          "enabled": False
        },
        "coordinate": {
          "enabled": False
        }
      },
      "layerBlending": "normal",
      "splitMaps": [],
      "animationConfig": {
        "currentTime": None,
        "speed": 1
      }
    },
    "mapState": {
      "bearing": 0,
      "dragRotate": False,
      "latitude": 44.85169239146265,
      "longitude": -0.5868239240658858,
      "pitch": 0,
      "zoom": 11.452871077625481,
      "isSplit": False
    },
    "mapStyle": {
      "styleType": "muted",
      "topLayerGroups": {
        "water": False
      },
      "visibleLayerGroups": {
        "label": True,
        "road": True,
        "border": False,
        "building": True,
        "water": True,
        "land": True,
        "3d building": False
      },
      "threeDBuildingColor": [
        137,
        137,
        137
      ],
      "mapStyles": {}
    }
  }
}




# VERSION GLOBAL
# "mapState": {
#      "bearing": 0,
#      "dragRotate": False,
#      "latitude": 44.85169239146265,
#      "longitude": -0.5868239240658858,
#      "pitch": 0,
#      "zoom": 11.452871077625481,
#      "isSplit": False
#    },

### Graphique

In [22]:
# Load kepler.gl with map data and config
map_kepler = KeplerGl(height=500, data={"data_1": station_control_kepler}, config=config_global)
map_kepler

User Guide: https://docs.kepler.gl/docs/keplergl-jupyter


KeplerGl(config={'version': 'v1', 'config': {'visState': {'filters': [], 'layers': [{'id': 'fmdzqhw', 'type': …

In [23]:
station_control_kepler

,station_id,mean_activity,is_anomaly,is_inactive,available_bikes,lat,lon,NOM,etat,anomaly_since_str,etat_id_sort
3,993,0.00,0,0,22,60.961662,-39.152723,Station non surveillée,non surveillée,-,0
31,158,0.05,0,0,12,44.858580,-0.668345,Le Haillan Rostand,non surveillée,-,0
32,148,0.05,0,0,26,44.885965,-0.541447,Berges de Garonne,non surveillée,-,0
33,79,0.05,0,0,6,44.786200,-0.638740,Pessac Bersol,non surveillée,-,0
34,95,0.05,0,0,3,44.881450,-0.613370,Bruges Hotel de Ville,non surveillée,-,0
...,...,...,...,...,...,...,...,...,...,...,...
90,137,0.21,0,0,30,44.828327,-0.562299,Place Andre Meunier,normal,-,1
1,59,0.42,1,0,9,44.849780,-0.570240,CAPC,anomaly,2021-04-07 10:10,2
0,6,0.40,1,0,3,44.837790,-0.581660,Square Andre Lhote,anomaly,2021-04-07 09:00,2
0,990,0.40,1,0,35,60.961662,-39.152723,Station fake anomaly,anomaly,2021-04-07 09:00,2


In [24]:
map_kepler.save_to_html(file_name='keplergl_map_global.html')

Map saved to keplergl_map_global.html!


In [23]:
# To select a station
station_id_to_plot = 6

# Get lat / lon
station_id_to_plot_lat = \
    station_control_kepler[station_control_kepler['station_id'] == station_id_to_plot]['lat'].values[0]
station_id_to_plot_lon = \
    station_control_kepler[station_control_kepler['station_id'] == station_id_to_plot]['lon'].values[0]

In [24]:
# Custom view for station plot
config_station = config_global
config_station['config']['mapState']['latitude'] = station_id_to_plot_lat
config_station['config']['mapState']['longitude'] = station_id_to_plot_lon
config_station['config']['mapState']['zoom'] = 15.3
config_station['config']['mapState']['bearing'] = 24
config_station['config']['mapState']['dragRotate'] = True
config_station['config']['mapState']['pitch'] = 54
config_global['config']['mapStyle']['visibleLayerGroups']['3d building'] = True # Building 3D

In [25]:
# Load kepler.gl with map data and config for One Station
map_kepler = KeplerGl(height=400, data={"data_1": station_control_kepler}, config=config_station)
map_kepler

User Guide: https://docs.kepler.gl/docs/keplergl-jupyter


KeplerGl(config={'version': 'v1', 'config': {'visState': {'filters': [], 'layers': [{'id': 'fmdzqhw', 'type': …

In [26]:
map_kepler.save_to_html(file_name='keplergl_map_station.html')

Map saved to keplergl_map_station.html!


# Industrialisation

## Simulation de données

In [31]:
import pandas as pd
import numpy as np
import io

from vcub_keeper.reader.reader import read_stations_attributes
from vcub_keeper.config import ROOT_DATA_REF

# Simulation de données

simulated_data="""station_id,mean_activity,is_anomaly,is_inactive,last_date_anomaly,anomaly_since
6,0.4,1,0,2021-04-07 11:10:00+02:00,2021-04-07 09:00:00+02:00
59,0.42,1,0,2021-04-07 11:10:00+02:00,2021-04-07 10:10:00+02:00
138,0.36,0,1,,
181,0.0,0,0,,
181,0.0,0,0,,
160,0.0,0,0,,
161,0.01,0,0,,
180,0.01,0,0,,
92,0.01,0,0,,
167,0.01,0,0,,
168,0.01,0,0,,
183,0.01,0,0,,
81,0.02,0,0,,
150,0.02,0,0,,
72,0.02,0,0,,
71,0.02,0,0,,
91,0.02,0,0,,
80,0.02,0,0,,
165,0.02,0,0,,
156,0.02,0,0,,
157,0.03,0,0,,
87,0.03,0,0,,
182,0.03,0,0,,
93,0.03,0,0,,
94,0.03,0,0,,
149,0.04,0,0,,
82,0.04,0,0,,
88,0.05,0,0,,
164,0.05,0,0,,
76,0.05,0,0,,
77,0.05,0,0,,
158,0.05,0,0,,
148,0.05,0,0,,
79,0.05,0,0,,
95,0.05,0,0,,
151,0.05,0,0,,
86,0.06,0,0,,
153,0.06,0,0,,
159,0.06,0,0,,
85,0.06,0,0,,
162,0.06,0,0,,
169,0.07,0,0,,
177,0.07,0,0,,
78,0.07,0,0,,
147,0.07,0,0,,
97,0.07,0,0,,
84,0.08,0,0,,
163,0.08,0,0,,
63,0.09,0,0,,
146,0.09,0,0,,
166,0.1,0,0,,
89,0.1,0,0,,
152,0.1,0,0,,
143,0.1,0,0,,
115,0.1,0,0,,
142,0.11,0,0,,
64,0.11,0,0,,
74,0.12,0,0,,
144,0.12,0,0,,
51,0.12,0,0,,
114,0.13,0,0,,
145,0.13,0,0,,
83,0.13,0,0,,
113,0.13,0,0,,
251,0.13,0,0,,
38,0.13,0,0,,
70,0.14,0,0,,
75,0.14,0,0,,
175,0.14,0,0,,
178,0.16,0,0,,
32,0.16,0,0,,
12,0.17,0,0,,
90,0.17,0,0,,
50,0.17,0,0,,
49,0.18,0,0,,
155,0.18,0,0,,
129,0.18,0,0,,
52,0.18,0,0,,
26,0.18,0,0,,
31,0.18,0,0,,
29,0.19,0,0,,
53,0.2,0,0,,
30,0.2,0,0,,
154,0.2,0,0,,
35,0.2,0,0,,
126,0.2,0,0,,
176,0.21,0,0,,
25,0.21,0,0,,
117,0.21,0,0,,
122,0.21,0,0,,
137,0.21,0,0,,
96,0.22,0,0,,
73,0.23,0,0,,
141,0.23,0,0,,
27,0.23,0,0,,
69,0.23,0,0,,
14,0.23,0,0,,
179,0.23,0,0,,
47,0.24,0,0,,
13,0.24,0,0,,
46,0.24,0,0,,
116,0.24,0,0,,
48,0.24,0,0,,
67,0.25,0,0,,
121,0.25,0,0,,
173,0.25,0,0,,
17,0.26,0,0,,
140,0.27,0,0,,
107,0.27,0,0,,
171,0.27,0,0,,
33,0.27,0,0,,
61,0.27,0,0,,
34,0.27,0,0,,
170,0.28,0,0,,
132,0.28,0,0,,
128,0.28,0,0,,
119,0.28,0,0,,
15,0.29,0,0,,
18,0.29,0,0,,
124,0.3,0,0,,
118,0.3,0,0,,
111,0.31,0,0,,
10,0.31,0,0,,
62,0.32,0,0,,
66,0.32,0,0,,
105,0.32,0,0,,
130,0.32,0,0,,
112,0.32,0,0,,
2,0.32,0,0,,
68,0.32,0,0,,
23,0.34,0,0,,
11,0.34,0,0,,
60,0.34,0,0,,
120,0.34,0,0,,
9,0.34,0,0,,
3,0.35,0,0,,
45,0.35,0,0,,
136,0.36,0,0,,
24,0.36,0,0,,
21,0.36,0,0,,
108,0.37,0,0,,
16,0.37,0,0,,
131,0.37,0,0,,
36,0.38,0,0,,
20,0.39,0,0,,
110,0.39,0,0,,
139,0.39,0,0,,
42,0.4,0,0,,
125,0.4,0,0,,
19,0.41,0,0,,
135,0.41,0,0,,
41,0.43,0,0,,
28,0.44,0,0,,
7,0.44,0,0,,
109,0.44,0,0,,
4,0.44,0,0,,
57,0.44,0,0,,
55,0.44,0,0,,
37,0.45,0,0,,
8,0.46,0,0,,
58,0.47,0,0,,
99,0.48,0,0,,
134,0.48,0,0,,
104,0.49,0,0,,
40,0.49,0,0,,
98,0.49,0,0,,
172,0.5,0,0,,
1,0.51,0,0,,
56,0.51,0,0,,
43,0.51,0,0,,
44,0.53,0,0,,
100,0.54,0,0,,
133,0.55,0,0,,
103,0.58,0,0,,
101,0.6,0,0,,
102,0.63,0,0,,
5,0.64,0,0,,
65,0.65,0,0,,
174,0.67,0,0,,
123,0.74,0,0,,
54,0.74,0,0,,
39,0.82,0,0,,
22,0.82,0,0,,
127,0.9,0,0,,
106,1.03,0,0,,
"""

station_control = pd.read_csv( io.StringIO(simulated_data)  , sep=",")

# Idem as read_station_control_from_s3 from vcub_watcher functino
station_control['last_date_anomaly'] = \
        pd.to_datetime(station_control['last_date_anomaly'])
try:
    station_control['last_date_anomaly'] = \
        station_control['last_date_anomaly'].dt.tz_localize('Europe/Paris')
except:  # try to convert TZ
    station_control['last_date_anomaly'] = \
        station_control['last_date_anomaly'].dt.tz_convert('Europe/Paris')

#station_control['anomaly_since'] = \
#    pd.to_datetime(station_control['anomaly_since'], utc=True)
station_control['anomaly_since'] = \
    pd.to_datetime(station_control['anomaly_since'])
try:
    station_control['anomaly_since'] = \
        station_control['anomaly_since'].dt.tz_localize('Europe/Paris')
except:  # try to convert TZ
    station_control['anomaly_since'] = \
        station_control['anomaly_since'].dt.tz_convert('Europe/Paris')
    
# Simulate "available_bikes" - Nombre de vélo présent à la station
station_control['available_bikes'] = np.random.randint(0, 40, len(station_control))

In [32]:
# Ajouter le lat/lon à station_control
# Lecture du fichier de réf des stations
stations = read_stations_attributes(path_directory=ROOT_DATA_REF)
stations = stations[['station_id', 'lat', 'lon', 'NOM']]

# Merge des données
station_control = station_control.merge(stations, on='station_id', how='left')

# On ne prend que les stations avec lat / lon
station_control = station_control[~station_control.lat.isna()]

In [33]:
station_control.head()

,station_id,mean_activity,is_anomaly,is_inactive,last_date_anomaly,anomaly_since,available_bikes,lat,lon,NOM
0,6,0.40,1,0,2021-04-07 11:10:00+02:00,2021-04-07 09:00:00+02:00,31,44.837790,-0.581660,Square Andre Lhote
1,59,0.42,1,0,2021-04-07 11:10:00+02:00,2021-04-07 10:10:00+02:00,20,44.849780,-0.570240,CAPC
3,181,0.00,0,0,NaT,NaT,11,44.866214,-0.605290,Mairie du Bouscat
4,181,0.00,0,0,NaT,NaT,10,44.866214,-0.605290,Mairie du Bouscat
5,160,0.00,0,0,NaT,NaT,37,44.912045,-0.724596,Eglise St Aubin


## Avec plotly

In [34]:
from vcub_keeper.config import MAPBOX_TOKEN

In [35]:
def plot_map_station_with_plotly(station_control,
                                 station_id=None,
                                 offline_plot=False):
    """
    Affiche une cartographie de l'agglomération de Bordeaux avec toutes les stations Vcub et leurs états
    provenant des algorithmes (normal, inactive et anomaly).

    Si "station_id" est indiqué, alors la cartographie est focus sur la lat / lon de station (Numéro)

    Parameters
    ----------
    data : pd.DataFrame
        En provenance de station_control.csv (vcub_watcher)
    station_id : Int [opt]
        Numéro de station que l'on souhaite voir (en focus) sur la cartographie.
    offline_plot : bool [opt]
        Pour retourner le graphique et l'utiliser dans une application

    Returns
    -------
    None

    Examples
    --------

    plot_map_station_with_plotly(station_control=station_control, offline_plot=False)
    """
    # Param plot with a given station_id
    if station_id is not None:
        # On centre le graphique sur la lat / lon de la station
        center_lat = \
            station_control[station_control['station_id'] == station_id]['lat'].values[0]
        center_lon = \
            station_control[station_control['station_id'] == station_id]['lon'].values[0]
        zoom_plot = 15
    else:
        center_lat = 44.837794
        center_lon = -0.581662
        zoom_plot = 11

    # Preprocess avant graphique
    station_control['etat'] = 'normal'

    # Non monitoré
    station_control.loc[station_control['mean_activity'] < THRESHOLD_PROFILE_STATION,
                        'etat'] = 'non surveillée'

    # En anoamlie (HS prediction)
    station_control.loc[station_control['is_anomaly'] == 1, 'etat'] = 'anomaly'

    # Inactive
    station_control.loc[station_control['is_inactive'] == 1, 'etat'] = 'inactive'

    # Transform date to string
    station_control['anomaly_since_str'] = \
        station_control['anomaly_since'].dt.strftime(date_format='%Y-%m-%d %H:%M')
    station_control['anomaly_since_str'] = station_control['anomaly_since_str'].fillna('-')

    # Color for etat
    color_etat = {'anomaly': '#EB4D50',
                  'inactive': '#5E9BE6',
                  'normal': '#6DDE75',
                  'non surveillée': '#696A6A'}

    # To know when use add_trace after init fig
    wtf_compteur = 0

    for etat in station_control['etat'].unique():
        # Filter
        temp = station_control[station_control['etat'] == etat]

        # Building text
        texts = []
        for idx, station in temp.iterrows():
            text = str(station['NOM']) \
                + " <br />" + "station N° : " + str(station['station_id']) \
                + " <br />" + "Nombre de vélo dispo : " + str(station['available_bikes']) \
                + " <br />" + "Activité suspecte depuis : " + str(station['anomaly_since_str'])
            texts.append(text)

        if wtf_compteur == 0:
            fig = go.Figure(go.Scattermapbox(lat=temp['lat'],
                                             lon=temp['lon'],
                                             mode='markers',
                                             hoverinfo='text',
                                             hovertext=texts,
                                             marker_size=9,
                                             marker_color=color_etat[etat],
                                             name=etat))
        else:
            fig.add_trace(go.Scattermapbox(lat=temp['lat'],
                                           lon=temp['lon'],
                                           mode='markers',
                                           hoverinfo='text',
                                           hovertext=texts,
                                           marker_size=9,
                                           marker_color=color_etat[etat],
                                           name=etat))

        wtf_compteur = 1

    fig.update_layout(mapbox=dict(center=dict(lat=center_lat, lon=center_lon),
                                  accesstoken=MAPBOX_TOKEN,
                                  zoom=zoom_plot,
                                  style="light"),
                      showlegend=True,
                      legend=dict(orientation="h",
                                  yanchor="top",
                                  xanchor="center",
                                  y=1.1,
                                  x=0.5
                                  ))
    if offline_plot is False:
        iplot(fig)
    else:
        offline.plot(fig)

In [36]:
from vcub_keeper.visualisation import plot_map_station_with_plotly

In [ ]:
plot_map_station_with_plotly(station_control=station_control, offline_plot=False)

In [ ]:
plot_map_station_with_plotly(station_control=station_control, offline_plot=False, station_id=6)

## Avec Kepler

In [39]:
def plot_map_station_with_kepler(station_control, station_id=None):
    """
    Affiche une cartographie de l'agglomération de Bordeaux avec toutes les stations Vcub et leurs états
     provenant des algorithmes (normal, inactive et anomaly).

    Si "station_id" est indiqué, alors la cartographie est focus sur la lat / lon de station (Numéro)

    Parameters
    ----------
    data : pd.DataFrame
        En provenance de station_control.csv (vcub_watcher)
    station_id : Int [opt]
        Numéro de station que l'on souhaite voir (en focus) sur la cartographie.
    Returns
    -------
    map_kepler : Graphique

    Examples
    --------

    map_kepler = plot_map_station_with_kepler(data=station_control, station_id=6)
    """

    # Global config plot
    config_global = {
      "version": "v1",
      "config": {
        "visState": {
          "filters": [],
          "layers": [
            {
              "id": "fmdzqhw",
              "type": "point",
              "config": {
                "dataId": "data_1",
                "label": "Station Vcub",
                "color": [
                  18,
                  147,
                  154
                ],
                "columns": {
                  "lat": "lat",
                  "lng": "lon",
                  "altitude": None
                },
                "isVisible": True,
                "visConfig": {
                  "radius": 12,
                  "fixedRadius": False,
                  "opacity": 0.8,
                  "outline": False,
                  "thickness": 2,
                  "strokeColor": None,
                  "colorRange": {
                    "name": "Custom Palette",
                    "type": "custom",
                    "category": "Custom",
                    "colors": [
                      "#696A6A",
                      "#6DDE75",
                      "#EB4D50",
                      "#5E9BE6"
                    ]
                  },
                  "strokeColorRange": {
                    "name": "Global Warming",
                    "type": "sequential",
                    "category": "Uber",
                    "colors": [
                      "#5A1846",
                      "#900C3F",
                      "#C70039",
                      "#E3611C",
                      "#F1920E",
                      "#FFC300"
                    ]
                  },
                  "radiusRange": [
                    0,
                    50
                  ],
                  "filled": True
                },
                "hidden": False,
                "textLabel": [
                  {
                    "field": None,
                    "color": [
                      255,
                      255,
                      255
                    ],
                    "size": 18,
                    "offset": [
                      0,
                      0
                    ],
                    "anchor": "start",
                    "alignment": "center"
                  }
                ]
              },
              "visualChannels": {
                "colorField": {
                  "name": "etat_id_sort",
                  "type": "integer"
                },
                "colorScale": "ordinal",
                "strokeColorField": None,
                "strokeColorScale": "quantile",
                "sizeField": None,
                "sizeScale": "linear"
              }
            }
          ],
          "interactionConfig": {
            "tooltip": {
              "fieldsToShow": {
                "data_1": [
                  {
                    "name": "station_id",
                    "format": None
                  },
                  {
                    "name": "NOM",
                    "format": None
                  },
                  {
                    "name": "etat",
                    "format": None
                  },
                  {
                    "name": "available_bikes",
                    "format": None
                  },
                  {
                    "name": "anomaly_since_str",
                    "format": None
                  }
                ]
              },
              "compareMode": True,
              "compareType": "absolute",
              "enabled": True
            },
            "brush": {
              "size": 0.5,
              "enabled": False
            },
            "geocoder": {
              "enabled": False
            },
            "coordinate": {
              "enabled": False
            }
          },
          "layerBlending": "normal",
          "splitMaps": [],
          "animationConfig": {
            "currentTime": None,
            "speed": 1
          }
        },
        "mapState": {
          "bearing": 0,
          "dragRotate": False,
          "latitude": 44.85169239146265,
          "longitude": -0.5868239240658858,
          "pitch": 0,
          "zoom": 11.452871077625481,
          "isSplit": False
        },
        "mapStyle": {
          "styleType": "muted",
          "topLayerGroups": {
            "water": False
          },
          "visibleLayerGroups": {
            "label": True,
            "road": True,
            "border": False,
            "building": True,
            "water": True,
            "land": True,
            "3d building": False
          },
          "threeDBuildingColor": [
            137,
            137,
            137
          ],
          "mapStyles": {}
        }
      }
    }

    # Preprocess avant graphique
    station_control['etat'] = 'normal'

    # Non monitoré
    station_control.loc[station_control['mean_activity'] < THRESHOLD_PROFILE_STATION,
                        'etat'] = 'non surveillée'

    # En anoamlie (HS prediction)
    station_control.loc[station_control['is_anomaly'] == 1, 'etat'] = 'anomaly'

    # Inactive
    station_control.loc[station_control['is_inactive'] == 1, 'etat'] = 'inactive'

    # Transform date to string
    station_control['anomaly_since_str'] = \
        station_control['anomaly_since'].dt.strftime(date_format='%Y-%m-%d %H:%M')
    station_control['anomaly_since_str'] = station_control['anomaly_since_str'].fillna('-')

    # Drop date for Kepler
    station_control = station_control.drop(['last_date_anomaly', 'anomaly_since'], axis=1)

    # Add fake stations to have every type of etat (anomaly / normal / inactive
    # & non surveillée) to match with fill color in plot

    fake_station_every_etat = \
        [[990, 0.4, 1, 0, 35, 60.9616624, -39.1527227, 'Station fake anomaly',
            'anomaly', '2021-04-07 09:00'],
         [991, 0.42, 1, 0, 21, 60.9616624, -39.1527227, 'Station fake normal',
          'normal', '-'],
         [992, 0.36, 0, 1, 22, 60.9616624, -39.1527227, 'Station fake inactive',
            'inactive', '-'],
         [993, 0.0, 0, 0, 22, 60.9616624, -39.1527227, 'Station non surveillée',
            'non surveillée', '-']]

    fake_station_every_etat_df = pd.DataFrame(fake_station_every_etat, columns=station_control.columns)
    station_control = pd.concat([station_control, fake_station_every_etat_df])

    # Sorting DataFrame to fill color in correct order
    etat_id_sort = {'non surveillée': 0,
                    'normal': 1,
                    'anomaly': 2,
                    'inactive': 3}

    station_control['etat_id_sort'] = station_control['etat'].map(etat_id_sort)
    station_control = station_control.sort_values('etat_id_sort')

    station_control = station_control.fillna("")

    # Param plot with a given station_id
    if station_id is not None:
        # On centre le graphique sur la lat / lon de la station
        center_lat = \
            station_control[station_control['station_id'] == station_id]['lat'].values[0]
        center_lon = \
            station_control[station_control['station_id'] == station_id]['lon'].values[0]

        config_global['config']['mapState']['latitude'] = center_lat
        config_global['config']['mapState']['longitude'] = center_lon
        config_global['config']['mapState']['zoom'] = 15.3
        config_global['config']['mapState']['bearing'] = 24
        config_global['config']['mapState']['dragRotate'] = True
        config_global['config']['mapState']['pitch'] = 54
        # Building 3D
        config_global['config']['mapStyle']['visibleLayerGroups']['3d building'] = True
        # file_name = 'keplergl_map_station.html'
    else:
        # file_name = 'keplergl_map_global.html'
        pass

    # Load kepler.gl with map data and config
    map_kepler = KeplerGl(height=400, data={"data_1": station_control}, config=config_global)

    # Export
    # map_kepler.save_to_html(file_name=file_name) # No more export
    return map_kepler

In [40]:
from vcub_keeper.visualisation import plot_map_station_with_kepler

In [41]:
map_kepler = plot_map_station_with_kepler(station_control=station_control)
map_kepler

User Guide: https://docs.kepler.gl/docs/keplergl-jupyter


KeplerGl(config={'version': 'v1', 'config': {'visState': {'filters': [], 'layers': [{'id': 'fmdzqhw', 'type': …

In [43]:
map_kepler = plot_map_station_with_kepler(station_control=station_control, station_id=6)
map_kepler

User Guide: https://docs.kepler.gl/docs/keplergl-jupyter


KeplerGl(config={'version': 'v1', 'config': {'visState': {'filters': [], 'layers': [{'id': 'fmdzqhw', 'type': …